# Agentes de IA do Azure com Suporte ao Protocolo de Contexto de Modelo (MCP)

Este notebook demonstra como utilizar os Agentes de IA do Azure com ferramentas do Protocolo de Contexto de Modelo (MCP) para criar um agente inteligente que pode aproveitar servidores MCP externos para capacidades aprimoradas.


## Instalar Pacotes NuGet Necessários

Primeiro, precisamos instalar o pacote Azure AI Agents Persistent, que fornece a funcionalidade principal para trabalhar com Azure AI Agents.


## Benefícios da Autenticação Sem Chave

Este notebook demonstra a **autenticação sem chave**, que oferece várias vantagens:
- ✅ **Sem necessidade de gerir chaves de API** - Utiliza autenticação baseada em identidade do Azure
- ✅ **Segurança melhorada** - Nenhum segredo armazenado no código ou na configuração
- ✅ **Rotação automática de credenciais** - O Azure gere o ciclo de vida das credenciais
- ✅ **Acesso baseado em funções** - Utiliza o RBAC do Azure para permissões detalhadas

O `DefaultAzureCredential` usará automaticamente a melhor fonte de credenciais disponível:
1. Identidade Gerida (quando executado no Azure)
2. Credenciais do Azure CLI (durante o desenvolvimento)
3. Credenciais do Visual Studio
4. Variáveis de ambiente (se configuradas)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Instale o pacote Azure Identity para autenticação com serviços Azure usando DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Importar os Namespaces Necessários

Importe os namespaces necessários para Azure AI Agents e Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Configurar o Cliente Azure AI Agent (Autenticação Sem Chave)

Configure as variáveis de configuração e crie o PersistentAgentsClient utilizando **autenticação sem chave**:
- **projectEndpoint**: O endpoint do projeto Azure AI Foundry
- **modelDeploymentName**: O nome do modelo de IA implementado (GPT-4.1 nano)
- **mcpServerUrl**: O URL do servidor MCP (API Microsoft Learn)
- **mcpServerLabel**: Uma etiqueta para identificar o servidor MCP
- **DefaultAzureCredential**: Utiliza identidade gerida, Azure CLI ou outras fontes de credenciais (não são necessárias chaves de API)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Criar Definição de Ferramenta MCP

Crie uma definição de ferramenta MCP que se conecte ao servidor MCP da Microsoft Learn. Isto permitirá que o agente aceda ao conteúdo e à documentação da Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Criar o Agente de IA

Crie um agente de IA persistente com o modelo e as ferramentas MCP especificados. O agente está configurado com:
- O modelo GPT-4.1 nano
- Instruções para utilizar as ferramentas MCP como suporte
- Acesso ao servidor Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Criar Tópico e Enviar Mensagem

Crie um tópico de conversa e envie uma mensagem ao utilizador a perguntar sobre a diferença entre Azure OpenAI e OpenAI. Isto irá testar a capacidade do agente de usar as ferramentas MCP para fornecer informações precisas.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Configurar Recursos da Ferramenta MCP (Sem Chave)

Configure os recursos da ferramenta MCP. Para uma abordagem verdadeiramente sem chave, pode remover os cabeçalhos personalizados se o servidor MCP suportar autenticação baseada em identidade Azure. O exemplo abaixo mostra como adicionar cabeçalhos, caso seja necessário, mas para cenários sem chave, estes podem não ser necessários.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Iniciar Execução do Agente

Crie e inicie uma execução para processar a mensagem do utilizador. O agente utilizará as ferramentas e recursos MCP configurados para gerar uma resposta.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Monitorizar a Execução e Gerir Aprovações de Ferramentas (Sem Chave)

Monitorize o estado de execução do agente e trate de quaisquer aprovações de ferramentas necessárias. Este ciclo:
1. Aguarda que a execução seja concluída ou que seja necessária uma ação
2. Aprova automaticamente chamadas de ferramentas MCP quando necessário
3. Para autenticação sem chave, os cabeçalhos podem não ser necessários se o servidor MCP suportar identidade Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Exibir Resultados da Conversa

Recupere e exiba todas as mensagens na conversa, mostrando tanto a pergunta do utilizador como a resposta do agente. As mensagens são exibidas em ordem cronológica com marcas de tempo e indicadores de função.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Aviso de Responsabilidade**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, é importante notar que traduções automáticas podem conter erros ou imprecisões. O documento original na sua língua nativa deve ser considerado a fonte autoritária. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas decorrentes do uso desta tradução.
